In [1]:
import matplotlib.pyplot as pyplot
%matplotlib inline

In [3]:
# loading scripts are in scripts/dataset.py
from scripts import dataset
import chainer
import os

# example code from tests/test_snapshot.py
dataset_name = 'lsp'
#dataset_name = 'lsp'

if dataset_name == 'mpii':
    DATASET_ROOT = os.path.expanduser('~/data/mpii')  # MPII dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(MPII_DATASET_ROOT, 'images')
    symmetric_joints = "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -100500
else:
    DATASET_ROOT = os.path.expanduser('~/data/lsp')  # LSP dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, '/test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(DATASET_ROOT, 'images')
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1

test_dataset = dataset.PoseDataset(
        TEST_CV_FILEPATH,
        IMG_PATH_PREFIX, 227,
        fliplr=False, rotate=False,
        shift=None,
        bbox_extension_range=(1.0, 1.0),
        coord_normalize=True,
        gcn=True,
        fname_index=0,
        joint_index=1,
        symmetric_joints=symmetric_joints,
        ignore_label=ignore_label,
        should_return_bbox=True,
        should_downscale_images=True,
        downscale_height=400
)

ModuleNotFoundError: No module named 'cv2'